### Importing the necessary libraries

In [39]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd 
import numpy as np
import json

#### Defining the url from where the data is to be scrapped

In [2]:
url = "https://www.daft.ie"

rent_url = 'https://www.daft.ie/property-for-rent/ireland'

#### Defining 'Headers'

For this you should first find your user agent which needs to be passed as a parameter for defining the headers. To find your user agent, go to 'https://www.whatismybrowser.com/detect/', under 'Detect Web Browser', you'll find 'What is my user agent?'. You have to copy that user agent of yours and pass it as a value against 'User-Agent' parameter in the below code snippet.

In [4]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 'Accept-Language': 'en-US', 'en':'q=0.5'}


#### Defining the http request

In [10]:
webpage = requests.get(rent_url,headers = header)

Print the response code:

In [11]:
webpage

<Response [200]>

#### Fetch all the html data of the website from where data is to be scrapped

In [13]:
soup = bs(webpage.content, "html.parser")

Now that we have fetched all the html data, we need to extract all the links (anchor tags) from this data

In [14]:
links = soup.find_all("a",attrs={'class':'SearchPagestyled__StyledA-sc-az4xxz-16 clQvUb'})

Now we'll be storing all these links in a list. So let's define an empty list to do the same:

In [15]:
links_list = []

Loop for extracting the links from from the html tag objects

In [18]:
for link in links:
    links_list.append(link.get('href'))

We'll be storing all the property data in a dictionary. So let's define the dictionary structure:

In [37]:
d = {"title":[],"description":[],"property_facilities":[],"statistics":[],"daft_id":[]}

#### Now let's define the functions to extract property details from each property listing url:

i) Function for extracting property name

In [27]:

def get_title(soup):

    try:
        title = soup.find("h1",attrs={'data-testid':'alt-title'}).text.strip()

    except AttributeError:
        title = ""
    
    return title

ii) Function for extracting property description

In [28]:
def get_description(soup):

    try:
        desc = soup.find("div",attrs={'data-testid':'alt-info'}).text.strip()

    except AttributeError:
        desc = ""
    
    return desc

iii) Function for extracting property facilites

In [32]:
def get_facilities(soup):

    try:
        property_facilities = soup.find("div",attrs={'data-testid':'facilities'}).find("ul",attrs={'class':'PropertyDetailsList__PropertyDetailsListContainer-sc-v5wteg-0 fLOYJA'}).text

    except AttributeError:
        property_facilities = ""
    
    return property_facilities

iv) Function for extracting statistics like date and property views

In [34]:
def get_statistics(soup):

    try:
        statistics = soup.find("div",attrs={'class':'Statistics__MainContainer-sc-49r9ge-4 khRNKJ'}).text

    except AttributeError:
        statistics = ""
    
    return statistics

v) Function for extracting the Daft ID

In [33]:
def get_id(soup):

    try:
        daft_id = soup.find("p",attrs={'class':'DaftIDText__StyledDaftIDParagraph-sc-4tzyji-0 kdpQoe'}).text.strip()

    except AttributeError:
        daft_id = ""

    return daft_id


Loop for extracting property listings from each link

In [42]:
for link in links_list:
    new_webpage = requests.get(url + link,headers = header)

    new_soup = bs(new_webpage.content, "html.parser")

    # Now let us call all the above the functions to fetch the required details accordingly against each of the parameters
    d['title'].append(get_title(new_soup))
    d['description'].append(get_description(new_soup))
    d['property_facilities'].append(get_facilities(new_soup))
    d['statistics'].append(get_statistics(new_soup))
    d['daft_id'].append(get_id(new_soup))

In [43]:
daft_df = pd.DataFrame.from_dict(d)
daft_df['title'].replace('',np.nan, inplace=True)
daft_df = daft_df.dropna(subset=['title'])



In [53]:
daft_df.to_csv("daft.csv",header=True , index=False)

In [44]:
daft_df

,title,description,property_facilities,statistics,daft_id
0,"The Cornerstone, Lower Kilmacud Road, Stillorg...","3 Property TypesFrom €2,050 per monthNew Devel...",ParkingCentral HeatingWashing MachineDryerDish...,"22/08/2024Entered/Renewed28,619Property Views",Daft ID: 919264787
1,"Cathedral Court, Unit 6 New Street South, Dubl...","1 Property TypesFrom €5,000 per monthNew Devel...",ParkingAlarmCable TelevisionWashing MachineDry...,"24/08/2024Entered/Renewed124,010Property Views",Daft ID: 93894824
2,"Occu East Village, Leopardstown, Dublin 18, Cl...","1 Property TypesFrom €2,150 per monthNew Devel...",ParkingCentral HeatingWashing MachineDryerDish...,"24/08/2024Entered/Renewed84,179Property Views",Daft ID: 916999292
3,"Niche Living, Private Studios, ALL Bills INCL,...","2 Property TypesFrom €1,895 per monthNew Devel...",Central HeatingAlarmCable TelevisionWashing Ma...,"24/08/2024Entered/Renewed22,093Property Views",Daft ID: 919485611
4,"Quayside Quarter, Dublin Landings, North Wall ...","4 Property TypesFrom €2,630 per monthNew Devel...",ParkingCentral HeatingCable TelevisionWashing ...,"24/08/2024Entered/Renewed1,445,824Property Views",Daft ID: 9162471
5,"Newmarket Yards, Dublin 2","4 Property TypesFrom €2,400 per monthNew Devel...",Central HeatingCable TelevisionWashing Machine...,"24/08/2024Entered/Renewed84,336Property Views",Daft ID: 918778912
6,"Balroy Hall, Carpenterstown Road, Castleknock,...","3 Property TypesFrom €1,950 per monthNew Devel...",ParkingCentral HeatingCable TelevisionWashing ...,"24/08/2024Entered/Renewed25,291Property Views",Daft ID: 919734883
7,"St Edmunds Avenue , St Edmunds Avenue , Lucan,...","2 Property TypesFrom €2,000 per monthNew Devel...",Central HeatingCable TelevisionWashing Machine...,"24/08/2024Entered/Renewed288,139Property Views",Daft ID: 914808034
8,"One Lime Street, Docklands, Dublin 2","2 Property TypesFrom €2,500 per monthNew Devel...",Washing MachineDryerDishwasherMicrowavePets Al...,"24/08/2024Entered/Renewed138,554Property Views",Daft ID: 915355502
9,"Green Acre Grange , Upper Kilmacud Road , Dund...","3 Property TypesFrom €2,050 per monthNew Devel...",ParkingCentral HeatingAlarmCable TelevisionWas...,"24/08/2024Entered/Renewed121,908Property Views",Daft ID: 918643399
